# Figure

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat
        library(Seurat), 
        
        # Condiment
        library(condiments), 
        library(tradeSeq), 
        library(SingleCellExperiment),
        library(clusterExperiment), 
        
        # GSEA
        library(fgsea), 
        library(msigdbr), 
        
        # Data 
        library(tidyverse), 
        library(reshape2), 
        library(tibble), 
        
        # Plot 
        library(ggplot2), 
        library(ComplexUpset), 
        library(circlize), 
        library(viridis), 
        library(ComplexHeatmap), 
        library(ggplotify), 
        library(gridExtra)
        
    )
)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
options(warn=-1)

In [ ]:
# Source files
source("plotting_global.R")
source("bin/tradeseq.R")
source("bin/seurat_qc.R")
source("bin/seurat_dea.R")
source("bin/cell_type.R")
source("bin/gene_modules.R")

In [ ]:
ht_opt$message=FALSE

In [ ]:
so_file <- "data/object/pp.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set(1))

# Import data

In [ ]:
so <- readRDS(so_file)

In [ ]:
table(so$treatment)

In [ ]:
# Set meta data factor level for plotting 
so$treatment <- factor(so$treatment, levels=names(color$treatment))
so$tissue <- factor(so$tissue, levels=names(color$tissue))
so$cell_type_fine <- factor(so$cell_type_fine, levels=names(color$cell_type_fine))
so$cell_type_fine_detail <- factor(so$cell_type_fine_detail, levels=names(color$cell_type_fine_detail))

## UMAP cell types 

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
source("bin/seurat_qc.R")
dplot_1 <- dplot(so, reduction="umap", group_by="cell_type_fine_detail", alpha=1, pt_size=2) + 
    scale_color_manual(values=color[["cell_type_fine_detail"]]) + 
    theme(
        
        legend.title=element_blank(), 
        plot.title=element_blank()
        
    ) + 
    
    guides(color=guide_legend(override.aes=list(alpha=1, size=5), ncol=2, keywidth=0.75, keyheight=2))

dplot_1

In [ ]:
ggsave("result/figures/umap_cell_type_fine.pdf", dplot_1, width=30, height=30, units="cm", dpi=1200)

## UMAP treatment

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

dplot_1 <- dplot(subset(so, subset=treatment=="NaCl"), reduction="umap", group_by="treatment", alpha=1, pt_size=2) + 
    scale_color_manual(values=color[["treatment"]]) + 
    theme(
        legend.title=element_blank(), 
        plot.title=element_blank()
    )

dplot_2 <- dplot(subset(so, subset=treatment=="CpG"), reduction="umap", group_by="treatment", alpha=1, pt_size=2) + 
    scale_color_manual(values=color[["treatment"]]) + 
    theme(
        legend.title=element_blank(), 
        plot.title=element_blank()
    )

dplot_1 + dplot_2

In [ ]:
ggsave("result/figures/umap_treatment_nacl.pdf", dplot_1, width=10, height=10, units="cm", dpi=1200)

## Cell type composition 

In [ ]:
options(repr.plot.width=2.5, repr.plot.height=3)

bplot_1 <- ggplot(so@meta.data, aes(x=treatment, fill=forcats::fct_rev(cell_type_fine_detail))) + 
    geom_bar(aes(x=factor(treatment, level=c("NaCl", "CpG"))), position="fill", width=0.75, border=NA) + 
    ggtitle("") + xlab("") + ylab("Cell type [rel.]") + 
    scale_fill_manual(values=color[["cell_type_fine_detail"]], name="") + 
    guides(fill=guide_legend(override.aes=list(alpha=1, size=2), ncol=2, keywidth=1, keyheight=1)) + 
    scale_y_continuous(expand=c(0, 0.01)) + 
    theme(legend.position="none")

bplot_1

In [ ]:
ggsave("result/figures/bar_cell_type_composition.pdf", bplot_1, width=7.5, height=20, units="cm", dpi=1200)

## Marker gene Dot plot 

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
source("bin/gene_modules.R")
genes_marker_figure$cell_type <- factor(genes_marker_figure$cell_type, levels=c("Ery", "MegP", "HSC", "MastP", "NeuP", "EoP", "BasoP",  "GMP/MDP", "RPM", "cDC2", "cDC1"))

so$cell_type_fine <- factor(so$cell_type_fine, levels=cell_type_fine_order)

dp_1 <- suppressMessages(dp_feature(so, genes_marker_figure$genes, genes_marker_figure$cell_type, group_by="cell_type_fine", scale=TRUE, range_max=7, title="")) + 
    theme_global_set() + 
    theme(
        legend.position="bottom",  
        legend.key.height=unit(0.5, "cm"), 
        legend.key.width=unit(0.5, "cm"), 
        axis.text.y=element_text(size=12, face="plain", margin=margin(t=0, r=2, b=0, l=0), color="black"),
        axis.text.x=element_text(size=12, face="plain", margin=margin(t=2, r=0, b=0, l=0), color="black"),
        strip.text=element_text(size=14, face="bold", margin=margin(t=2, r=2, b=2, l=2), color="black")
    )

dp_1

In [ ]:
ggsave("result/figures/dp_marker_genes.pdf", dp_1, width=24, height=24, units="cm", dpi=1200)

## Absorption probability 

In [ ]:
# CellRank absorption probabilities 
absorption_probabilities_prog_nacl <- read.csv("result/cellrank/absorption_probabilities_prog_nacl.csv", row.names=1, check.names=FALSE)
absorption_probabilities_prog_cpg <- read.csv("result/cellrank/absorption_probabilities_prog_cpg.csv", row.names=1,check.names=FALSE)

so <- AddMetaData(so, rbind(absorption_probabilities_prog_nacl, absorption_probabilities_prog_cpg))

In [ ]:
data <- so@meta.data[so$cell_type_main %in% c("MEP", "ProEB", "EB"), ] %>% 
    select(treatment, cell_type_fine, EB..5.) %>% 
    reshape2::melt(id_vars=c("treatment", "cell_type_fine")) %>% as.data.frame() %>% 
    dplyr::mutate(cell_type_fine=factor(cell_type_fine, levels=cell_type_prog)) %>% 
    dplyr::mutate(treatment=factor(treatment, levels=c("NaCl", "CpG"))) %>% 
    dplyr::mutate(cell_type_fine=droplevels(cell_type_fine)) %>% 
    dplyr::group_by(treatment)

In [ ]:
data_summary <- function(data, varname, groupnames){
    suppressMessages(require(plyr))
    summary_func <- function(x, col) {
      
      c(
          mean=mean(x[[col]], na.rm=TRUE),
          sd=sd(x[[col]], na.rm=TRUE)
      )
  }
    data_sum<-plyr::ddply(data, groupnames, .fun=summary_func, varname)
    data_sum <- rename(data_sum, c("mean"=varname))
    return(data_sum)
}

In [ ]:
data <- data_summary(data, varname="value", groupnames=c("treatment", "cell_type_fine"))

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

plot <- ggplot(data, aes(x=cell_type_fine, y=value, color=treatment, fill=treatment, group=treatment)) + 
    geom_errorbar(aes(ymin=value-sd, ymax=value+sd), width=0.5, size=unit(2/2.141959, "pt")) +
    geom_line(size=unit(2/2.141959, "pt")) + geom_point(size=3) + 
    scale_color_manual(values=unlist(color[["treatment"]])) + 
    
    theme(
    
        axis.text.x=element_text(face="plain", color="black", angle=90, vjust=0.5, hjust=1),
        axis.text.y=element_text(face="plain", color="black", vjust=0.5, hjust=1),

        legend.position="bottom"
        
    ) 

plot

In [ ]:
ggsave("result/figures/absorption_probability_eb.pdf", plot, width=15, height=13, units="cm", dpi=1200)

# Erythroid lineage 

## Forced atlas layout with diffusion pseudotime 

In [ ]:
dpt_pseudotime <- read.csv(paste0("result/cellrank/dpt_pseudotime_eb.csv"), row.names=1)
so_eb <- subset(so, subset=cell_id %in% rownames(dpt_pseudotime))
so_eb <- AddMetaData(so_eb, dpt_pseudotime)
so_eb[["fag"]] <- CreateDimReducObject(embeddings=rbind(as.matrix(read.csv(paste0("result/cellrank/fag_eb_nacl.csv"), row.names=1)), as.matrix(read.csv(paste0("result/cellrank/fag_eb_cpg.csv"), row.names=1))), key="FAG_")

In [ ]:
dpt_pseudotime_nacl <- subset(so_eb, subset=treatment=="NaCl")@meta.data[, "dpt_pseudotime", drop=FALSE]
dpt_pseudotime_nacl$dpt_pseudotime <- scales::rescale(dpt_pseudotime_nacl$dpt_pseudotime)

dpt_pseudotime_cpg <- subset(so_eb, subset=treatment=="CpG")@meta.data[, "dpt_pseudotime", drop=FALSE]
dpt_pseudotime_cpg$dpt_pseudotime <- scales::rescale(dpt_pseudotime_cpg$dpt_pseudotime)

In [ ]:
so_eb <- AddMetaData(so_eb, rbind(dpt_pseudotime_nacl, dpt_pseudotime_cpg))

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

source("bin/seurat_qc.R")

fplot_1 <- fplot(so_eb, subset=c("treatment", "NaCl"), reduction="fag", feature="dpt_pseudotime", alpha=1, pt_size=3, slot="data") + ggtitle("Steady state")
fplot_2 <- fplot(so_eb, subset=c("treatment", "CpG"), reduction="fag", feature="dpt_pseudotime", alpha=1, pt_size=3, slot="data") + ggtitle("Stress")

fplot_1 + fplot_2 + plot_layout(ncol=2)

In [ ]:
ggsave("result/figures/fag_dpt_eb_nacl.pdf", fplot_1, width=15, height=15, units="cm", dpi=1200)
ggsave("result/figures/fag_dpt_eb_cpg.pdf", fplot_2, width=15, height=15, units="cm", dpi=1200)

## Heatmap of Pseudotime associated genes (PTAG)

In [ ]:
tradeseq_eb <- readRDS("result/tradeseq/tradeseq_eb.rds")
ptg_eb <- readRDS("result/tradeseq/ptg_eb.rds")

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=7.5)

genes <- rownames(ptg_eb[ptg_eb$ptag_nacl==TRUE & ptg_eb$ptag_cpg==TRUE & ptg_eb$ptcg==FALSE, ])

n = 100
width = 2.5*(50/n)

hm_ptag <- hm_smooth(tradeseq_eb[["fitgam"]], genes=genes, n=n, cluster_rows=TRUE, show_row_names=FALSE, width=width, height=0.075)
hm_ptag

In [ ]:
ggsave("result/figures/hm_tradeseq_ptag_eb.pdf", hm_ptag, width=10, height=6, units="cm", dpi=1200)

In [ ]:
grep("38a", rownames(so), value=TRUE)

## Smoothed expression of marker genes

In [ ]:
genes <- c("Kit", "Cd47", "Gata1", "Gata2", "Epor", "Tfrc", "Klf1", "Tal1", "Stat1", "Stat5a", "Rb1")